In [9]:
from LoadingBMWDataset import ObjectDetectionDataset
from sklearn.model_selection import train_test_split
import torch
import albumentations as A
import json
import os
from PIL import Image
import shutil

In [2]:
def convert_annotations_to_yolo_format(img_dir, annotation_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists

    # Iterating over each annotation file in the annotation directory, and convert the annotations to YOLO format
    for annotation_file in os.listdir(annotation_dir):
        if annotation_file.endswith('.json'):
            json_path = os.path.join(annotation_dir, annotation_file)
            with open(json_path) as f:
                annotations = json.load(f)
            
            # Deriving the corresponding image file path and load it to get dimensions
            img_file = annotation_file.replace('.json', '.jpg')
            img_path = os.path.join(img_dir, img_file)
            with Image.open(img_path) as img:
                img_width, img_height = img.size
            
            # Converting annotations to YOLO format
            yolo_annotations = []
            for annot in annotations:
                class_id = annot['ObjectClassId'] - 1  # Assuming class IDs start from 1, adjust if necessary
                x_center = ((annot['Right'] + annot['Left']) / 2) / img_width
                y_center = ((annot['Bottom'] + annot['Top']) / 2) / img_height
                width = (annot['Right'] - annot['Left']) / img_width
                height = (annot['Bottom'] - annot['Top']) / img_height
                yolo_annotations.append(f"{class_id} {x_center} {y_center} {width} {height}")
            
            # Saving converted annotations to TXT file
            txt_path = os.path.join(output_dir, annotation_file.replace('.json', '.txt'))
            with open(txt_path, 'w') as f:
                f.write('\n'.join(yolo_annotations))


In [3]:
img_dir = '/home/wgt/Desktop/InMind Academy/AI_Track/Amazing_Project/inmind_amazing_project/data/Training/images'
annotation_dir = '/home/wgt/Desktop/InMind Academy/AI_Track/Amazing_Project/inmind_amazing_project/data/Training/labels/json'
output_dir = '/home/wgt/Desktop/InMind Academy/AI_Track/Amazing_Project/inmind_amazing_project/data/Training/labels/yolo'

convert_annotations_to_yolo_format(img_dir, annotation_dir, output_dir)

In [4]:
from sklearn.model_selection import train_test_split
import glob

annotations_txt_dir = '/home/wgt/Desktop/InMind Academy/AI_Track/Amazing_Project/inmind_amazing_project/data/Training/labels/yolo'
img_dir = '/home/wgt/Desktop/InMind Academy/AI_Track/Amazing_Project/inmind_amazing_project/data/Training/images'

# Listing all .txt annotation files
annotation_files = glob.glob(os.path.join(annotations_txt_dir, '*.txt'))

# Extracting corresponding image file names from annotation file names
img_files = [os.path.join(img_dir, os.path.basename(f).replace('.txt', '.jpg')) for f in annotation_files]

# Splitting the dataset into training and validation
train_img_files, val_img_files, train_annotation_files, val_annotation_files = train_test_split(img_files, annotation_files, test_size=0.2, random_state=42)


In [6]:
print(f"Number of training images: {len(train_img_files)}")


Number of training images: 1995


### Moving to YOLOV7

In [ ]:
yolov7_train_images_dir = '/home/wgt/yolov7/train/images'
yolov7_train_labels_dir = '/home/wgt/yolov7/train/labels'

os.makedirs(yolov7_train_images_dir, exist_ok=True)
os.makedirs(yolov7_train_labels_dir, exist_ok=True)

for img_file in train_img_files:
    dest_file = os.path.join(yolov7_train_images_dir, os.path.basename(img_file))
    shutil.copy(img_file, dest_file)
    print(f"Copied {img_file} to {dest_file}")

for label_file in train_annotation_files:
    dest_file = os.path.join(yolov7_train_labels_dir, os.path.basename(label_file))
    shutil.copy(label_file, dest_file)
    print(f"Copied {label_file} to {dest_file}")

yolov7_val_images_dir = '/home/wgt/yolov7/val/images'
yolov7_val_labels_dir = '/home/wgt/yolov7/val/labels'

os.makedirs(yolov7_val_images_dir, exist_ok=True)
os.makedirs(yolov7_val_labels_dir, exist_ok=True)

for img_file in val_img_files:
    dest_file = os.path.join(yolov7_val_images_dir, os.path.basename(img_file))
    shutil.copy(img_file, dest_file)
    print(f"Copied {img_file} to {dest_file}")

for label_file in val_annotation_files:
    dest_file = os.path.join(yolov7_val_labels_dir, os.path.basename(label_file))
    shutil.copy(label_file, dest_file)
    print(f"Copied {label_file} to {dest_file}")

## Results of the first Training process:

<br>Class|||||Images||||Labels|||||||||||P||||||||||||R||||||||||mAP@.5|||mAP@.5:.95 </br>
<br>  all||||||||||483|||||||||1995|||||||||||0.96|||||||0.954|||||0.976||||0.919 </br>
<br>  dolly|||||||483|||||||||847||||||||||||0.934||||||0.913|||||0.957||||0.855 </br>
<br>  bin|||||||||483|||||||||349||||||||||||0.961||||||0.983|||||0.992||||0.948 </br>
<br>  jack||||||||483|||||||||799||||||||||||0.986||||||0.967|||||0.98|||||0.952 </br>

<br> We can see that the precision is very high (96% overall), as well as the recall (95.4%). 
<br> The Mean Average Precision (AP) at IoU (Intersection over Union) threshold of 0.5 indicates a 97.6% meaning a very high accuracy.
<br> The the mean AP calculated at different IoU thresholds from 0.5 to 0.95 (with a step size of 0.05), provides a more comprehensive view of the model performance across various strictness levels of object detection.

<br> The model shows high precision and recall across the classes dolly, bin, and jack, with overall great mAP scores, which indicates strong performance in both recognizing the correct objects (high recall) and ensuring that most detections are accurate (high precision)
